In [1]:
import transformers
from datasets import load_dataset
import torch
import random

/root/anaconda3/envs/difficulty/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/mnt/cephfs/sumin/sentence_kd/sft/ckpts/s1-difficulty-before-think"
# model_path = "/mnt/cephfs/echoi/models/Qwen2.5-Math-7B/"
# model_path = "/mnt/cephfs/echoi/models/Qwen2.5-7B-Instruct/"
config = transformers.AutoConfig.from_pretrained(model_path)
# config.use_sliding_window = True
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_path,
        config=config,
        attn_implementation='flash_attention_2',
        torch_dtype=torch.bfloat16,
        device_map="auto" 
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_path
)

Loading checkpoint shards: 100%|██████████| 7/7 [00:57<00:00,  8.24s/it]


In [3]:
# dataset = load_dataset("HuggingFaceH4/MATH-500")
# dataset = load_dataset("garage-bAInd/Open-Platypus")
# dataset = load_dataset("Maxwell-Jia/AIME_2024")
dataset = load_dataset("KbsdJames/Omni-MATH")

Generating test split: 100%|██████████| 4428/4428 [00:00<00:00, 66980.83 examples/s]


In [4]:
dataset

DatasetDict({
    test: Dataset({
        features: ['domain', 'difficulty', 'problem', 'solution', 'answer', 'source'],
        num_rows: 4428
    })
})

In [14]:
random_index = random.randint(0, len(dataset['test']['problem']))
input_text = dataset['test']['problem'][89]


In [15]:
input_text

'Find the smallest positive integer $ K$ such that every $ K$-element subset of $ \\{1,2,...,50 \\}$ contains two distinct elements $ a,b$ such that $ a\\plus{}b$ divides $ ab$.'

In [16]:
from transformers import StoppingCriteria, StoppingCriteriaList
import re

class FinalAnswerStoppingCriteria(StoppingCriteria):
    def __init__(self, tokenizer, prompt_len):
        super().__init__()
        self.tokenizer = tokenizer
        self.pattern = re.compile(r"\[\s*Q\s*\]")  # 정확히 [QED] 패턴 매칭
        self.prompt_len = prompt_len  # 프롬프트 길이 저장

    def __call__(self, input_ids, scores, **kwargs):
        # 현재까지 생성된 전체 시퀀스에서 프롬프트 이후의 부분만 추출
        generated_ids = input_ids[0][self.prompt_len:]
        decoded = self.tokenizer.decode(generated_ids, skip_special_tokens=True)

        return bool(self.pattern.search(decoded))


In [17]:
system_prompt = """
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
"""
prefix_prompt = """
"Rate the difficulty of this problem on a scale of 1-10 and include it at the beginning of your response. Example: Difficulty: 5/10"
"""
def add_prefix_to_instruction(instruction):
    return prefix_prompt + "\n\n Q: " + instruction + "\n\n"
def add_suffix_to_instruction(instruction):
    return "Q: " + instruction + "\n" + prefix_prompt + "\n\n"
prompted_instruction = add_prefix_to_instruction(input_text)
message = [
    {"role": "system", "content": system_prompt + "\n" + prefix_prompt},
    {"role": "user", "content": input_text}
]

text = tokenizer.apply_chat_template(
    message,
    tokenize = False,
    add_generation_prompt = True
)
model_inputs = tokenizer([text], return_tensors='pt').to('cuda')

stop_criteria = StoppingCriteriaList([
    FinalAnswerStoppingCriteria(tokenizer, model_inputs['input_ids'].shape[1])
])
output = model.generate(
                **model_inputs,
                # attention_mask = inputs['attention_mask'],
                max_new_tokens=512,
                # early_stopping=True,
                stopping_criteria=stop_criteria,
                do_sample=True,
                num_beams=1,
                eos_token_id=None,
            )

In [18]:
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
def format_response_with_line_breaks(response):
    # Print the response with line breaks properly displayed
    print(response)
    
    # If you need to save it to a file with proper formatting
    with open("formatted_response.txt", "w", encoding="utf-8") as f:
        f.write(response)
    
    # If you're in a Jupyter notebook, you can use display for HTML formatting
    from IPython.display import display, HTML
    formatted_html = response.replace("\n", "<br>")
    display(HTML(f"<div>{formatted_html}</div>"))
from IPython.display import Markdown
def display_with_formatting(text):
    display(Markdown(text))
# format_response_with_line_breaks(output_text)
display_with_formatting(output_text)

system

You are Qwen, created by Alibaba Cloud. You are a helpful assistant.


"Rate the difficulty of this problem on a scale of 1-10 and include it at the beginning of your response. Example: Difficulty: 5/10"

user
Find the smallest positive integer $ K$ such that every $ K$-element subset of $ \{1,2,...,50 \}$ contains two distinct elements $ a,b$ such that $ a\plus{}b$ divides $ ab$.
assistant
Difficulty: 7/10
answer
Let $S = \{1, 2, ..., 50\}$. We are looking for the smallest positive integer $K$ such that every $K$-element subset of $S$ contains two distinct elements $a, b$ such that $a+b$ divides $ab$.

The condition $a+b \mid ab$ is equivalent to $\frac{ab}{a+b} = \frac{ab}{a+b} \cdot \frac{a-b}{a-b} = \frac{a^2 - b^2}{a+b} = \frac{a^2}{a+b}$ if $a > b$, or $\frac{b^2}{a+b}$ if $b > a$. This does not seem to simplify the problem much.

Let's consider the condition $a+b \mid ab$.
If $a+b \mid ab$, then $a+b \mid (a+b) \cdot \frac{a}{a+b} \cdot (a+b) = a^2 + ab - ab + b^2 = a^2 + b^2$.
So $a+b \mid a^2+b^2$.

We can rewrite the condition as $ab = k(a+b)$ for some integer $k$.
$ab - ka - kb = 0$
$ab - ka - kb + k^2 = k^2$
$(a-k)(b-k) = k^2$.

Let $a-k = x$ and $b-k = y$, so $xy = k^2$, $a = x+k$, $b = y+k$.
Since $a, b \in S$, we have $1 \le k+x \le 50$ and $1 \le k+y \le 50$.

Consider the case where $a=2x$ and $b=x$. Then $a+b = 3x$ and $ab = 2x^2$. The condition is $3x \mid 2x^2$.
$2x^2 = 2 \cdot x \cdot x = 3 \cdot x \cdot x$.
If $x$ is even, let $x=2m$, then $2(2m)^2 = 8m^2$, $3(2m) = 6m$. $8m^2 / 6m = 4m

In [33]:
dataset['test']['level'][1]

5